In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!pwd


Mounted at /content/drive
/content


In [ ]:
import os
os.chdir(r'/content/drive/MyDrive/Colab Notebooks/机器学习二大作业/')

os.getcwd()

'/content/drive/MyDrive/Colab Notebooks/机器学习二大作业'

In [ ]:
from torch import nn 
import torch 
from torch.utils.data import Dataset,DataLoader
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams["font.sans-serif"]="SimHei"
plt.rcParams["axes.unicode_minus"]=True
fig_path="../latex/tex/figures/"
%matplotlib inline

In [ ]:
data=pd.read_csv("data_1.csv")
## 把Gender,Subject 等 构造成 Category
categorys=["Gender","SKIN","SPORT","Activity","Subject"]
for category in categorys:
    data[category]=data[category].astype("category")
X_columns=[i for i in data.columns if i!="Label"]
y_columns=["Label"]
X=data.loc[:,X_columns]
y=data.loc[:,y_columns]
display(X,y)

,chest_ACC_x,chest_ACC_y,chest_ACC_z,chest_Resp,chest_ECG,wrist_ACC_x,wrist_ACC_y,wrist_ACC_z,wrist_BVP,wrist_TEMP,WEIGHT,Gender,AGE,HEIGHT,SKIN,SPORT,Rpeaks,Activity,Subject
0,0.8564,-0.0678,-0.3656,4.441833,0.015610,-0.765625,-0.078125,0.671875,7.28,32.13,78.0,m,34,182.0,3,6,0,0,S1
1,0.8514,-0.0654,-0.3688,4.876709,-0.015747,-0.765625,-0.078125,0.671875,-11.36,32.16,78.0,m,34,182.0,3,6,1,0,S1
2,0.8514,-0.0662,-0.3708,3.340149,-0.008743,-0.765625,-0.078125,0.671875,-51.27,32.16,78.0,m,34,182.0,3,6,0,0,S1
3,0.8594,-0.0632,-0.3640,0.740051,-0.339523,-0.750000,-0.078125,0.671875,5.65,32.16,78.0,m,34,182.0,3,6,0,0,S1
4,0.8510,-0.0674,-0.3694,-1.475525,-0.089905,-0.765625,-0.078125,0.671875,34.34,32.16,78.0,m,34,182.0,3,6,0,0,S1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
517951,0.8878,0.0122,-0.2330,2.789307,-0.281845,-0.468750,0.859375,0.093750,1.68,33.99,79.0,m,28,183.0,2,5,0,0,S15
517952,0.8962,-0.0184,-0.2280,0.563049,0.075851,-0.609375,0.890625,0.265625,-16.38,33.99,79.0,m,28,183.0,2,5,1,0,S15
517953,0.9010,0.0006,-0.2102,-1.075745,0.017166,-0.468750,0.531250,-0.562500,13.43,33.99,79.0,m,28,183.0,2,5,0,0,S15
517954,0.8906,-0.0124,-0.2238,-2.117920,-0.304184,-0.375000,0.406250,-0.843750,-5.89,33.99,79.0,m,28,183.0,2,5,0,0,S15


,Label
0,49.611369
1,49.611369
2,49.611369
3,49.611369
4,49.611369
...,...
517951,79.664230
517952,79.664230
517953,79.664230
517954,79.664230


In [ ]:
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state = 42, stratify = y)
X_train=pd.DataFrame(X_train.to_numpy(),index=[i for i in range(X_train.shape[0])],columns=X.columns)
X_test=pd.DataFrame(X_test.to_numpy(),index=[i for i in range(X_test.shape[0])],columns=X.columns)
OneHot=OneHotEncoder()
X_train_cate=X_train.loc[:,categorys].copy()
X_test_cate=X_test.loc[:,categorys].copy()
X_train_trans=OneHot.fit_transform(X_train_cate).todense()
X_train_trans=pd.DataFrame(X_train_trans,index=[i for i in range(X_train_trans.shape[0])],columns=["cate"+str(i) for i in range(X_train_trans.shape[1])])
X_test_trans=OneHot.transform(X_test_cate).todense()
X_test_trans=pd.DataFrame(X_test_trans)
X_test_trans=X_test_trans.reindex_like(X_train_trans)
X_train=pd.merge(left=X_train.drop(categorys,axis=1),right=X_train_trans,left_index=True,right_index=True)
X_test=pd.merge(left=X_test.drop(categorys,axis=1),right=X_test_trans,left_index=True,right_index=True)
X_test.fillna(value=0,inplace=True)


X_train=StandardScaler().fit_transform(X_train)
X_test=StandardScaler().fit_transform(X_test)

In [ ]:
class LSTM(nn.Module):
    def __init__(self, input_size=48, hidden_layer_size=100, output_size=1,device=torch.device("cuda")):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size

        self.lstm = nn.LSTM(input_size, hidden_layer_size)

        self.linear = nn.Linear(hidden_layer_size, output_size)
        self.hidden_cell = (torch.zeros(1,1,self.hidden_layer_size).to(device),
                            torch.zeros(1,1,self.hidden_layer_size).to(device))

    def forward(self, input_seq):
        lstm_out, self.hidden_cell = self.lstm(input_seq.view(len(input_seq) ,1, -1), self.hidden_cell)
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
        return predictions

In [ ]:
## train_in_out_seq:
class MyData(Dataset):
    def __init__(self,X,y):
        if type(X)==np.ndarray:
            self.X=torch.tensor(X).to(torch.float32)
        else:
            self.X=torch.tensor(X.values).to(torch.float32)
        if type(y)==np.ndarray:
            self.y=torch.tensor(y).to(torch.float32)
        else:
            self.y=torch.tensor(y.values).to(torch.float32)
    def __len__(self):
        return len(self.y)
    def __getitem__(self,idx):
        return self.X[idx,:],self.y[idx]

batch_size=5000
train_loader=DataLoader(dataset=MyData(X_train,y_train),batch_size=batch_size)
test_loader=DataLoader(dataset=MyData(X_test,y_test),batch_size=batch_size)


In [ ]:
model = LSTM()
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [ ]:
epochs = 80
def train(model,epochs):
    model.train()
    device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(device)
    for i in range(epochs):
        tot_loss=0
        for data, labels in train_loader:
            optimizer.zero_grad()
            model.to(device)
            model.hidden_cell = (torch.zeros(1, 1, model.hidden_layer_size).to(device),
                        torch.zeros(1, 1, model.hidden_layer_size).to(device))
            y_pred = model(data.to(device)).to(device)
            
            single_loss = loss_function(y_pred, labels.to(device))
            single_loss.backward()
            optimizer.step()
            tot_loss+=single_loss*len(labels)/517956
        if i%10==0:
            print(f'epoch: {i:3} loss: {single_loss.item():10.8f}')
    print(f'epoch: {i:3} loss: {tot_loss.item():10.10f}')
def test(model):
    model.eval()
    tot_loss=0
    device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    for data,labels in test_loader:
        optimizer.zero_grad()
        model.to(device)
        model.hidden_cell = (torch.zeros(1, 1, model.hidden_layer_size).to(device),
                        torch.zeros(1, 1, model.hidden_layer_size).to(device))
        y_pred = model(data.to(device)).to(device)
        single_loss = loss_function(y_pred, labels.to(device))
            # single_loss.backward()
        optimizer.step()
        tot_loss+=single_loss*len(labels)/517956
    print(f"test loss:{tot_loss.item():10.10f}")



        

In [ ]:

train(model,epochs)


cuda
epoch:   0 loss: 8476.14453125
epoch:  10 loss: 6033.78027344
epoch:  20 loss: 4933.79003906
epoch:  30 loss: 4020.24511719
epoch:  40 loss: 3244.50463867
epoch:  50 loss: 2587.44970703
epoch:  60 loss: 2038.23010254
epoch:  70 loss: 1589.16052246
epoch:  79 loss: 948.0567016602


In [ ]:
test(model)

test loss:2.2786211967


In [ ]:
def r2_loss(output, target):
    """这个越大越好"""
    target_mean = torch.mean(target)
    ss_tot = torch.sum((target - target_mean) ** 2)
    # print(ss_tot)
    ss_res = torch.sum((target - output) ** 2)
    # print(ss_res)
    r2 = 1 - ss_res / ss_tot
    # print("ss_tot,ss_res,r2:",ss_tot,ss_res,r2)
    return r2

In [ ]:
def test(model):
    """r2_loss"""
    model.eval()
    tot_loss=0
    device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    for data,labels in test_loader:
        optimizer.zero_grad()
        model.to(device)
        model.hidden_cell = (torch.zeros(1, 1, model.hidden_layer_size).to(device),
                        torch.zeros(1, 1, model.hidden_layer_size).to(device))
        y_pred = model(data.to(device)).to(device)
        single_loss = r2_loss(y_pred, labels.to(device))
            # single_loss.backward()
        optimizer.step()
        tot_loss+=single_loss*len(labels)/517956
    print(f"test loss:{tot_loss.item():10.10f}")
test(model)

test loss:0.0417542160
